In [1]:
import os
from pypdf import PdfReader
import docx
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel

In [2]:
def get_data(path):
    all_content = []
    files = os.listdir(path)

    for file in files:
        pa = os.path.join(path,file)
        if pa.endswith('.docx'):
            doc = docx.Document(pa)
            para = doc.paragraphs
            content = "\n".join([i.text for i in para])
            all_content.append(content)
        elif pa.endswith('.pdf'):
            content = ""
            with open(pa,"rb") as f:
                pdf_reader = PdfReader(f)

                pages_info = pdf_reader.pages
                for page_info in pages_info:
                    text = page_info.extract_text()
                    content+=text
            all_content.append(content)
        elif pa.endswith('.txt'):
            with open(pa,'r',encoding='utf8') as f:
                all_data = f.read()
            all_content.append(all_data)
    return all_content

In [3]:
class DFaiss:
    def __init__(self):
        self.sentence_model = SentenceTransformer('./paraphrase-multilingual-MiniLM-L12-v2',cache_folder='sen_model')
        self.index = faiss.IndexFlatL2(384)
        self.text_str_list = []
    def add_content(self,strlist):
        text_emb = self.get_text_emb(strlist)
        self.text_str_list.extend(strlist)
        self.index.add(text_emb)
    def get_text_emb(self,strlist):
        return self.sentence_model.encode(strlist)
    def search(self,text):
        text_emb = self.get_text_emb([text])
        D,I = self.index.search(text_emb,3)
        if D[0][0]<15:
            return self.text_str_list[I[0][0]]
        else:
            return ""

In [4]:
class prompt_robot():
    def __init__(self):
        self.myfaiss = DFaiss()
        self.tokenizer = AutoTokenizer.from_pretrained("../THUDM/chatglm-6b", trust_remote_code=True)
        self.model = AutoModel.from_pretrained("../THUDM/chatglm-6b", trust_remote_code=True).half().cuda()
        self.model.eval()
    def ask(self,query):
        search_result = self.myfaiss.search(query)
        if len(search_result)==0:
            prompt = query
        else:
            prompt=f'请根据以下内容回答问题。内容是"{search_result}",问题是"{query}"。'
        print(f'the prompt is {prompt}')
        response, history = self.model.chat(self.tokenizer, prompt, history=[])
        return response
    def add_content(self,strlist):
        self.myfaiss.add_content(strlist)

In [5]:
if __name__ == "__main__":
    all_content = get_data('datas')
    robot = prompt_robot()
    robot.add_content(all_content)
    while True:
        print('----------------------------------')
        text = input('user:')
        print(f'robot:{robot.ask(text)}')

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

----------------------------------


user: zwk和浙江财经大学之间是什么关系？


The dtype of attention mask (torch.int64) is not bool


...

user: 中国平安保险在哪一年成立？


the prompt is 请根据以下内容回答问题。内容是"中国平安保险（集团）股份有限公司（以下简称“中国平安” ，“平安保险”，“中国平安保险公司”，“平安集团”）于1988年诞生于深圳蛇口，是中国第一家股份制保险企业，已经发展成为金融保险、银行、投资等金融业务为一体的整合、紧密、多元的综合金融服务集团。 [1] 
为香港联合交易所主板及上海证券交易所两地上市公司，股票代码分别为02318和601318。 [2] 
[74]  2019年《福布斯》全球2000强第1999位，较2018年排名跃升3位。蝉联全球多元保险企业第一，在中国众多入围企业中排名第5，全球金融企业排名第6；《2019年BrandZ最具价值全球品牌100强》公布，中国平安排名第40位；Brand Finance “2019全球品牌价值500强”榜单，中国平安品牌价值位列全球第14位。在“2019全球最具价值100大保险品牌（Brand Finance Insurance 100 2019）”排行榜中，中国平安荣列榜首； [3]  2019年9月1日，2019中国服务业企业500强榜单在济南发布，中国平安保险（集团）股份有限公司排名第3位。2019年12月，中国平安保险（集团）股份有限公司入选2019中国品牌强国盛典榜样100品牌。 [4]  2019年12月18日，人民日报发布中国品牌发展指数100榜单，中国平安排名第20位。 [5]  2020年3月，入选2020年全球品牌价值500强第9位。 [6]  2020年9月28日，入选2020中国企业500强榜单，排名第六。 [73]  2020年《财富》世界500强排行榜第21名，较2019年的29名跃升8名。2021年《财富》世界500强排名第16位 [77]  。

经营范围：投资保险企业；监督管理控股投资企业的各种国内、国际业务；开展保险资金运用业务；经批准开展国内、国际保险业务；经中国保险监督管理委员会及国家有关部门批准的其他业务。 [11] 















中国平安保险（集团）股份有限公司（以下简称“中国平安” ，“平安保险”，“中国平安保险公司”，“平安集团”）于1988年诞生于深圳蛇口，是中国第一家股份制保险企业，已经发展成为金融保险、银行、投资等金融业务为一体的整合、紧密、多元的综合金融服务集团。 [1] 
为香

user: 平安银行注册资金为多少？


the prompt is 请根据以下内容回答问题。内容是"平安银行，全称平安银行股份有限公司，是中国平安保险（集团）股份有限公司控股的一家跨区域经营的股份制商业银行，为中国大陆12家全国性股份制商业银行之一。注册资本为人民币51.2335亿元，总资产近1.37万亿元，总部位于广东省深圳市。
中国平安保险（集团）股份有限公司（以下简称“中国平安”）及其控股子公司持有平安银行股份共计约26.84亿股，占比约99.38%，为平安银行的控股股东。在全中国各地设有34家分行，在香港设有代表处。
2012年1月，现平安银行的前身深圳发展银行收购平安保险集团旗下的深圳平安银行，收购完成后，深圳发展银行更名为新的平安银行，组建新的平安银行正式对外营业。2019年6月26日，平安银行等8家银行首批上线运行企业信息联网核查系统。
",问题是"平安银行注册资金为多少？"。
robot:平安银行注册资金为人民币51.2335亿元。
----------------------------------


KeyboardInterrupt: Interrupted by user